In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# navigate to working folder
%cd '/content/drive/MyDrive/UIT/HK1_2023-2024/Project_01/Process'

/content/drive/MyDrive/UIT/HK1_2023-2024/Project_01/Process


In [3]:
!pwd

/content/drive/MyDrive/UIT/HK1_2023-2024/Project_01/Process


In [4]:
!ls

dataset				     model.png
emotion_model_base.h5		     model_trainer_self.ipynb
emotion_model_self.h5		     model_trainer_v01.ipynb
emotion_model_v01.h5		     model_trainer_v02.ipynb
emotion_model_v02.h5		     model_trainer_v03.ipynb
emotion_model_v03.h5		     model_trainer_v04.ipynb
emotion_model_v04.h5		     model_trainer_v05.ipynb
emotion_model_v05.h5		     model_trainer_v06.ipynb
emotion_model_v06.h5		     model_trainer_v07.ipynb
emotion_model_v07.h5		     model_trainer_v08.ipynb
emotion_model_v09.h5		     model_trainer_v09.ipynb
fer2013.csv			     model_trainer_v10_SOTA.ipynb
haarcascade_frontalface_default.xml  realtime_ed.py
kaggle.json


In [5]:
# import necessary libraries
import os
import cv2
from google.colab import files
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.utils import to_categorical
from keras.optimizers import SGD

In [ ]:
# load img func
def load_images_from_folder(folder):
    images = []
    labels = []
    label_mapping = {
        'angry': 0,
        'disgust': 1,
        'fear': 2,
        'happy': 3,
        'sad': 4,
        'surprise': 5,
        'neutral': 6
    }

    for emotion in os.listdir(folder):
        emotion_folder = os.path.join(folder, emotion)

        for filename in os.listdir(emotion_folder):
            img_path = os.path.join(emotion_folder, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                images.append(img)
                labels.append(label_mapping[emotion])
    return images, labels

In [ ]:
# download dataset .zip from kaggle

# Install Kaggle API client
!pip install -q kaggle

# Upload kaggle API key file using your local file
from google.colab import files
files.upload()

# Set up the Kaggle directory and API key file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Use Kaggle API to download dataset
# You need to replace 'DATASET_PATH' with the path of the dataset you want
!kaggle datasets download -d 'msambare/fer2013'

# For example, for the 'dogs vs cats' dataset, DATASET_PATH would be 'bikashjaiswal/dogs-vs-cats'


Saving kaggle.json to kaggle.json
 85% 51.0M/60.3M [00:00<00:00, 104MB/s]
100% 60.3M/60.3M [00:00<00:00, 101MB/s]


In [ ]:
# unzip the dataset

import zipfile
zip_path = "./dataset/fer2013.zip"  # replace with your downloaded file name
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("./dataset")

In [ ]:
# load images from train and test

    # train:
    #   - angry
    #   - disgust
    #   - fear
    #   - happy
    #   - sad
    #   - suprise
    #   - neutral
    # test:
    #   - angry
    #   - disgust
    #   - fear
    #   - happy
    #   - sad
    #   - suprise
    #   - neutral

train_images, train_labels = load_images_from_folder('./dataset/train')
test_images, test_labels = load_images_from_folder('./dataset/test')

In [ ]:
# convert the images and labels into a DataFrame
def create_dataframe(images, labels, usage):
    pixel_data = [' '.join(map(str, img.flatten())) for img in images]
    data = {'emotion': labels, 'pixels': pixel_data, 'Usage': usage}
    df = pd.DataFrame(data)
    return df

train_df = create_dataframe(train_images, train_labels, 'Training')
test_df = create_dataframe(test_images, test_labels, 'Test')

# Combine the data frames into one
final_df = pd.concat([train_df, test_df], ignore_index=True)


In [ ]:
# download csv file
final_df.to_csv('fer2013.csv', index=False)
files.download('fer2013.csv')

In [6]:
# Read the dataset into a pandas dataframe
df = pd.read_csv('./fer2013.csv')

In [7]:
print(df)

       emotion                                             pixels     Usage
0            0  131 133 131 131 132 138 155 150 134 148 141 16...  Training
1            0  118 118 142 165 168 160 165 169 167 168 167 16...  Training
2            0  83 87 88 89 78 96 108 102 119 128 147 162 157 ...  Training
3            0  32 37 45 46 61 67 80 89 89 94 99 107 111 113 1...  Training
4            0  245 227 218 226 221 223 240 235 236 231 225 16...  Training
...        ...                                                ...       ...
35882        5  239 241 240 239 241 238 227 127 124 161 156 15...      Test
35883        5  47 45 51 38 31 34 33 70 32 34 26 26 24 33 14 1...      Test
35884        5  68 79 89 102 117 125 127 127 128 136 140 138 1...      Test
35885        5  228 222 210 194 202 208 209 179 156 143 134 13...      Test
35886        5  46 46 41 48 55 55 62 63 39 33 30 44 45 45 58 5...      Test

[35887 rows x 3 columns]


In [8]:
# Preprocess data function
def preprocess_data(df):
    images = df["pixels"].apply(lambda x: np.array(x.split(" "), dtype=np.float32))
    images = np.array([x.reshape(48, 48, 1) for x in images])
    images /= 255.0
    labels = to_categorical(df["emotion"])
    return images, labels

In [9]:
# data splitting and data preprocessing

train_data = df[df["Usage"] == "Training"]
test_data = df[df["Usage"] == "Test"]

x_train, y_train = preprocess_data(train_data)
x_test, y_test = preprocess_data(test_data)

In [10]:
# model settings
model = Sequential([
    Conv2D(16, (3,3), activation='relu', input_shape=(48, 48, 1)),
    MaxPooling2D(2, 2),
    Conv2D(32, (3,3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(7, activation='softmax')
])

In [11]:
# compile the model (MUST DO BEFORE TRAINING)
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
history = model.fit(x_train, y_train, epochs=30, batch_size=64)

Epoch 1/30
449/449 [==============================] - 13s 6ms/step - loss: 1.6969 - accuracy: 0.3256
Epoch 2/30
449/449 [==============================] - 3s 7ms/step - loss: 1.4948 - accuracy: 0.4239
Epoch 3/30
449/449 [==============================] - 2s 5ms/step - loss: 1.3852 - accuracy: 0.4718
Epoch 4/30
449/449 [==============================] - 2s 5ms/step - loss: 1.2975 - accuracy: 0.5042
Epoch 5/30
449/449 [==============================] - 2s 5ms/step - loss: 1.2304 - accuracy: 0.5343
Epoch 6/30
449/449 [==============================] - 2s 5ms/step - loss: 1.1817 - accuracy: 0.5532
Epoch 7/30
449/449 [==============================] - 2s 5ms/step - loss: 1.1375 - accuracy: 0.5727
Epoch 8/30
449/449 [==============================] - 3s 7ms/step - loss: 1.0944 - accuracy: 0.5874
Epoch 9/30
449/449 [==============================] - 2s 5ms/step - loss: 1.0558 - accuracy: 0.6015
Epoch 10/30
449/449 [==============================] - 2s 5ms/step - loss: 1.0126 - accuracy: 0.620

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 16)        160       
                                                                 
 max_pooling2d (MaxPooling2  (None, 23, 23, 16)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 21, 21, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 10, 10, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 8, 8, 64)          18496     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 4, 4, 64)          0

In [14]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print("Loss: ", test_loss)
print("Test Accuracy: ", test_acc)

225/225 [==============================] - 1s 3ms/step - loss: 1.8990 - accuracy: 0.5588
Loss:  1.8989903926849365
Test Accuracy:  0.5587907433509827


In [15]:
model_name = 'emotion_model_self'
# save model
model.save(f"{model_name}.h5")
# Download the model
files.download(f"{model_name}.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>